<p>In this third phase, we will : </p> 
<ul> 
    <li> scrape and parse information about 10 hotels for every city from booking.com   </li> 
</ul>

### Table of Contents

* [1. Get cities and hotels' URLs](#section1)
* [2. Scrape and Parse hotels' data](#section2)

In [5]:
# data wrapping 
import csv

# scraping 
import requests
from selectorlib import Extractor

from tqdm import tqdm
from time import sleep

# predefined functions
from modules import Funct as F

# global params
bucket_name = 'kayak-project'

# 1. Get cities and hotels' URLs <a class="anchor" id="section1"></a>

In [16]:
# download data from data lake
F.download_file_dl(bucket_name, 'cities.txt', 'data/cities.txt')
F.download_file_dl(bucket_name, 'urls.txt', 'data/urls.txt')

File downloaded
File downloaded


In [7]:
cities = F.read_txt('data/cities.txt')
urls = F.read_txt('data/urls.txt')

Data retrieved
Data retrieved


# 2. Scrape and Parse hotels' data <a class="anchor" id="section2"></a>

In [8]:
def scrape(url, yml_file): 
    e = Extractor.from_yaml_file(yml_file)
    r = requests.get(url)
    data = e.extract(r.text,base_url=url)
    
    return data

In [ ]:
yml_file = 'data/booking.yml'
yml_geo_file = 'data/geo_data.yml'

success = len(urls)
with open('data/hotels.csv','w') as f:
    fieldnames = [
        "cid",
        "name",
        "star",
        "rating",
        "rating_title",
        "number_of_ratings",
        "price",
        "url",
        "lat",
        "lon"
    ]
    
    writer = csv.DictWriter(f, fieldnames = fieldnames, quoting = csv.QUOTE_NONNUMERIC)
    writer.writeheader()
    
    for i, url in tqdm(enumerate(urls)):
        data = scrape(url, yml_file) 
        if data['hotels']:
            for hotel in data['hotels']:
            
                hotel['cid'] = i
                
                # get geo coordinates of the hotel
                url_hotel = hotel['url']
                data_geo = scrape(url_hotel, yml_geo_file)
                lat, lon = data_geo['geo'][0]['address'].split(',')
                hotel['lat'] = lat
                hotel['lon'] = lon

                writer.writerow(hotel)
        else:
            print('No data found')
            success = success - 1
        sleep(5)
    print (f'{success} out of {len(urls)} success')

In [ ]:
# save hotels' data in the data Lake
F.upload_file_dl('data/hotels.csv', bucket_name, 'hotels.csv')         